In [1]:
DEFAULT_MODEL = "meta-llama/Llama-3.3-70B-Instruct"
N_OUTLINES = 5

In [2]:
from typing import Optional
import os
import torch
from accelerate import Accelerator
from transformers import AutoModelForCausalLM, AutoTokenizer

from tqdm.notebook import tqdm
import warnings

warnings.filterwarnings('ignore')

In [3]:
device = "cuda" if torch.cuda.is_available() else "cpu"

SYS_PROMPT = f"""
You are the world's best lead editor. You have 2 awards for breakfast and 3 awards for lunch.

Infact you are so good at this that you manage a team of not 1 but {N_OUTLINES} AI Agent researchers. 

The researchers work non-stop for you to come up with interesting outlines that you can then later send to your best editorial team

For now, you will get an open ended/vague or sometimes concrete theme or idea or topic to kickoff a research on

Your job is to first do one round of confirmation with humans since they are so inefficient at expressing thoughts as prompts

For step 1-you will confirm back to human what they want a deeply researched report on after that we will iterate on {N_OUTLINES} outlines that we want your AI agents to research

But after you receive confirmation, stop double confirming REMEMBER WE NNED TO MOVE Towards your dinner awards :D 
"""

In [4]:
accelerator = Accelerator()
model = AutoModelForCausalLM.from_pretrained(
    DEFAULT_MODEL,
    torch_dtype=torch.bfloat16,
    use_safetensors=True,
    device_map="auto",
)
tokenizer = AutoTokenizer.from_pretrained(DEFAULT_MODEL, use_safetensors=True)
model, tokenizer = accelerator.prepare(model, tokenizer)

Loading checkpoint shards:   0%|          | 0/30 [00:00<?, ?it/s]

In [5]:
conversation = [
        {"role": "system", "content": SYS_PROMPT},
        {"role": "user", "content": "I want to learn about the latest Llama model"},
    ]
    
prompt = tokenizer.apply_chat_template(conversation, tokenize=False)
inputs = tokenizer(prompt, return_tensors="pt").to(device)
    
with torch.no_grad():
    output = model.generate(
        **inputs,
        temperature=0.7,
        top_p=0.9,
        max_new_tokens=512
    )

generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
response = generated_text[len(prompt):]

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


In [6]:
conversation.append({"role": "assistant", "content": response})
print("Initial response:", response)

Initial response: of artificial intelligence model developed by Meta. You're looking for an in-depth analysis of its capabilities, features, and potential applications, is that correct? 

(Please confirm, and I'll proceed with instructing my team of 5 AI Agent researchers to come up with interesting outlines for the report)


In [7]:
user_input = """
1. I just want to know whats new in Llama 3.3 
2. How does it compare with 3.1 
3. Executive overview of its comparision with GPT-4o
"""

In [ ]:
conversation.append({"role": "user", "content": user_input})
prompt = tokenizer.apply_chat_template(conversation, tokenize=False)
inputs = tokenizer(prompt, return_tensors="pt").to(device)

with torch.no_grad():
    output = model.generate(**inputs, temperature=0.7, top_p=0.9, max_new_tokens=512)

generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
response = generated_text[len(prompt):]
conversation.append({"role": "assistant", "content": response})


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


In [ ]:
print(response)

In [ ]:
user_input = f"""
Alright Now we enter JSON ONLY RESPONSE mode, here's what you'll do now. Look at your chat history. 

After getting input from user, we now proceed to starting stating your AI Agent researchers ok?

JUST RESPOND in JSON the formatted response, the following details:
1. Report Title: This defines the angle of the report your AI Agent will write, your agents aren't as smart as you-you need to give them entire context below
2. Context: Give the dumb AI agents the context but its all worth it for your awards
3. SYSTEM_Personality: Here we define the personality of each AI Agent researcher, remember to vary them
4. Vibe: Fill the vibe to whatever you want
5. Energy: Define what energy do you want this AI Agent to achieve
6. Goal: Angle of the report

Alright, your time to shine-send off {N_OUTLINES}
"""

In [ ]:
conversation.append({"role": "user", "content": user_input})
prompt = tokenizer.apply_chat_template(conversation, tokenize=False)
inputs = tokenizer(prompt, return_tensors="pt").to(device)

with torch.no_grad():
    output = model.generate(**inputs, temperature=0.7, top_p=0.9, max_new_tokens=512)

generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
response = generated_text[len(prompt):]
conversation.append({"role": "assistant", "content": response})


In [ ]:
conversation